<a href="https://colab.research.google.com/github/movindugunarathna/sinXdetect/blob/main/ml/sinbert_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SinBERT Text Classifier for Sinhala AI Detection

This notebook implements a text classification model using NLPC-UOM's SinBERT-large model to detect AI-generated Sinhala text. SinBERT is specifically trained on Sinhala language data, making it potentially more effective for Sinhala text classification tasks compared to multilingual models.

## 1. Install Required Libraries

In [ ]:
%pip install -q tf-keras
%pip install -q transformers
%pip install -q datasets
%pip install -q nltk
%pip install -q scikit-learn
%pip install -q matplotlib
%pip install -q seaborn

## 2. Import Required Libraries

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    roc_auc_score,
    classification_report,
    precision_recall_curve,
    f1_score
)
import re
import unicodedata
import warnings
warnings.filterwarnings('ignore')

# Suppress TensorFlow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from transformers import (
    AutoTokenizer,
    TFAutoModelForSequenceClassification,
    AutoConfig
)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 3. Load Dataset from JSONL Files

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

def load_jsonl(filepath):
    """Load data from JSONL file"""
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return data

# Define the path to the dataset in Google Drive
# Update this path based on where you stored the 'dataset' folder in your Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/sinxdetect/dataset'

# Load training, validation, and test datasets
train_data = load_jsonl(os.path.join(drive_base_path, 'train.jsonl'))
val_data = load_jsonl(os.path.join(drive_base_path, 'val.jsonl'))
test_data = load_jsonl(os.path.join(drive_base_path, 'test.jsonl'))

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Testing set size: {len(test_data)}")

## 4. Convert to DataFrame and Explore Data

In [ ]:
# Convert to DataFrame
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Display sample data
print("Sample training data:")
print(train_df[['text', 'label']].head())
print(f"\nLabel distribution (Train):")
print(train_df['label'].value_counts())
print(f"\nLabel distribution (Validation):")
print(val_df['label'].value_counts())
print(f"\nLabel distribution (Test):")
print(test_df['label'].value_counts())

## 5. Map Labels to Numeric Values

In [ ]:
# Create label mapping
label_mapping = {'HUMAN': 0, 'AI': 1}
reverse_mapping = {v: k for k, v in label_mapping.items()}

# Map labels to numeric values
train_df['label_encoded'] = train_df['label'].map(label_mapping)
val_df['label_encoded'] = val_df['label'].map(label_mapping)
test_df['label_encoded'] = test_df['label'].map(label_mapping)

# Check for any unmapped values
print(f"Train - Unmapped labels: {train_df['label_encoded'].isna().sum()}")
print(f"Val - Unmapped labels: {val_df['label_encoded'].isna().sum()}")
print(f"Test - Unmapped labels: {test_df['label_encoded'].isna().sum()}")

## 6. Sinhala Text Preprocessing

**Important Note:** SinBERT was trained specifically on Sinhala text and its tokenizer is designed to handle Sinhala linguistic variations. We apply minimal preprocessing to preserve the natural text patterns that SinBERT was trained on. This is different from multilingual models that may benefit from more aggressive normalization.

Based on the word cloud analysis, common Sinhala words in the dataset include: මම, ඔබ, කළ, වන, සිංහල, කරන, etc. We preserve these natural forms.

In [ ]:
def preprocess_sinhala_text(text):
    """
    Minimal Sinhala text preprocessing optimized for SinBERT:
    - NFC Unicode normalization (essential for consistency)
    - Clean extra whitespace
    - Preserve natural Sinhala variations (SinBERT's tokenizer handles these)
    
    Note: We avoid aggressive normalization (contractions, zero-width chars, etc.) 
    because SinBERT was trained on natural Sinhala text with these variations.
    Removing them may hurt performance.
    """
    if not isinstance(text, str):
        return text

    # Normalize to NFC (Canonical Composition) - Essential for Unicode consistency
    text = unicodedata.normalize('NFC', text)

    # Clean extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove any unusual control characters (but keep common Sinhala ones)
    # Keep ZWNJ (U+200C) and ZWJ (U+200D) as they're part of proper Sinhala orthography
    text = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F-\x9F]', '', text)

    return text

# Apply preprocessing
print("Preprocessing training data...")
train_df['processed_text'] = train_df['text'].apply(preprocess_sinhala_text)

print("Preprocessing validation data...")
val_df['processed_text'] = val_df['text'].apply(preprocess_sinhala_text)

print("Preprocessing test data...")
test_df['processed_text'] = test_df['text'].apply(preprocess_sinhala_text)

print("\nPreprocessing complete!")
print(f"\nExample preprocessed text:")
print(f"Original: {train_df['text'].iloc[0][:100]}...")
print(f"Processed: {train_df['processed_text'].iloc[0][:100]}...")

## 7. Load SinBERT Tokenizer

We use `NLPC-UOM/SinBERT-large` which is specifically trained on Sinhala language data. This model should provide better performance for Sinhala text classification compared to multilingual models.

In [ ]:
# Load SinBERT tokenizer
MODEL_NAME = 'NLPC-UOM/SinBERT-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"✓ SinBERT tokenizer loaded: {MODEL_NAME}")
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Max length: {tokenizer.model_max_length}")

## 8. Tokenize and Encode Text Data

In [ ]:
# Configuration
MAX_LENGTH = 256  # Optimal for most Sinhala text
BATCH_SIZE = 32

def tokenize_in_batches(texts, tokenizer, batch_size=32, max_length=256):
    """
    Tokenize texts in batches to manage memory efficiently.
    Returns dict with 'input_ids' and 'attention_mask' as numpy arrays.
    """
    input_ids_parts = []
    attention_mask_parts = []

    total_batches = (len(texts) + batch_size - 1) // batch_size

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        # Tokenize batch
        enc = tokenizer(
            batch,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='tf'
        )

        # Convert to numpy
        ids = enc['input_ids'].numpy()
        mask = enc['attention_mask'].numpy()

        input_ids_parts.append(ids)
        attention_mask_parts.append(mask)

        if (i // batch_size + 1) % 10 == 0:
            print(f"  Processed {i // batch_size + 1}/{total_batches} batches")

    # Concatenate all batches
    input_ids = np.concatenate(input_ids_parts, axis=0)
    attention_mask = np.concatenate(attention_mask_parts, axis=0)

    return {'input_ids': input_ids, 'attention_mask': attention_mask}

# Tokenize datasets
print("Tokenizing training data...")
train_encodings = tokenize_in_batches(
    train_df['processed_text'].tolist(),
    tokenizer,
    batch_size=BATCH_SIZE,
    max_length=MAX_LENGTH
)

print("\nTokenizing validation data...")
val_encodings = tokenize_in_batches(
    val_df['processed_text'].tolist(),
    tokenizer,
    batch_size=BATCH_SIZE,
    max_length=MAX_LENGTH
)

print("\nTokenizing test data...")
test_encodings = tokenize_in_batches(
    test_df['processed_text'].tolist(),
    tokenizer,
    batch_size=BATCH_SIZE,
    max_length=MAX_LENGTH
)

print("\n✓ Tokenization complete!")
print(f"Training encodings shape: {train_encodings['input_ids'].shape}")
print(f"Validation encodings shape: {val_encodings['input_ids'].shape}")
print(f"Test encodings shape: {test_encodings['input_ids'].shape}")

## 9. Prepare Input Data and Labels

In [ ]:
# Prepare input dictionaries
train_inputs = {
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask']
}

val_inputs = {
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask']
}

test_inputs = {
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
}

# Prepare labels
train_labels = np.array(train_df['label_encoded'].astype(int).tolist())
val_labels = np.array(val_df['label_encoded'].astype(int).tolist())
test_labels = np.array(test_df['label_encoded'].astype(int).tolist())

print(f"Train labels shape: {train_labels.shape}")
print(f"Val labels shape: {val_labels.shape}")
print(f"Test labels shape: {test_labels.shape}")
print(f"\nLabel distribution (Train): {np.bincount(train_labels)}")
print(f"Label distribution (Val): {np.bincount(val_labels)}")
print(f"Label distribution (Test): {np.bincount(test_labels)}")

## 10. Load SinBERT Model for Sequence Classification

Note: SinBERT-large is originally trained as a Masked Language Model. We'll adapt it for sequence classification by loading the base model and adding a classification head.

In [ ]:
# Load SinBERT model configuration
config = AutoConfig.from_pretrained(MODEL_NAME)

# Update config for sequence classification
config.num_labels = 2  # Binary classification: HUMAN (0) vs AI (1)

# Load pre-trained SinBERT model for sequence classification
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config,
    from_pt=True  # Convert from PyTorch if needed
)

print(f"✓ SinBERT model loaded: {MODEL_NAME}")
print(f"Number of parameters: {model.num_parameters():,}")
print(f"Model type: {type(model).__name__}")

## 11. Compile the Model

In [ ]:
# Training configuration with regularization
LEARNING_RATE = 2e-5
EPOCHS = 3
TRAIN_BATCH_SIZE = 16

# Compile model
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    epsilon=1e-8,
    clipnorm=1.0  # Gradient clipping to prevent exploding gradients
)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True
)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[metric]
)

print("✓ Model compiled successfully with regularization!")
print(f"\nTraining Configuration:")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {TRAIN_BATCH_SIZE}")
print(f"  Max Length: {MAX_LENGTH}")
print(f"  Gradient Clipping: 1.0")

## 12. Set Up Callbacks

In [ ]:
# Create model directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Create callbacks for training
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=1,
        verbose=1,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='models/sinbert_checkpoint.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

print("✓ Callbacks configured:")
print("  - Early Stopping (patience=2)")
print("  - Learning Rate Reduction (factor=0.5, patience=1)")
print("  - Model Checkpoint (best validation accuracy)")

## 13. Train the SinBERT Model

In [ ]:
print("\n" + "="*60)
print("Starting Training...")
print("="*60 + "\n")

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

history = model.fit(
    train_inputs,
    train_labels,
    epochs=EPOCHS,
    batch_size=TRAIN_BATCH_SIZE,
    validation_data=(val_inputs, val_labels),
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("✓ Training Complete!")
print("="*60)

# Print training summary
print(f"\nTraining Summary:")
print(f"  Best Epoch: {np.argmax(history.history['val_accuracy']) + 1}")
print(f"  Best Val Accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"  Final Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"  Final Val Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"  Total Epochs Run: {len(history.history['accuracy'])}")

## 14. Save the Trained Model

In [ ]:
# Create model directory if it doesn't exist
MODEL_SAVE_PATH = "models/sinbert_sinhala_classifier/"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

# Save the model
model.save_pretrained(MODEL_SAVE_PATH)
print(f"✓ Model saved to {MODEL_SAVE_PATH}")

# Save the tokenizer
tokenizer.save_pretrained(MODEL_SAVE_PATH)
print(f"✓ Tokenizer saved to {MODEL_SAVE_PATH}")

# Save training configuration
config = {
    'model_name': MODEL_NAME,
    'max_length': MAX_LENGTH,
    'batch_size': TRAIN_BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'epochs': EPOCHS,
    'label_mapping': label_mapping
}

with open(os.path.join(MODEL_SAVE_PATH, 'training_config.json'), 'w') as f:
    json.dump(config, f, indent=2)

print(f"✓ Configuration saved to {MODEL_SAVE_PATH}training_config.json")

## 15. Evaluate on Validation Set

In [ ]:
# Evaluate on validation set
print("\nEvaluating on Validation Set...")
val_loss, val_accuracy = model.evaluate(val_inputs, val_labels, batch_size=32, verbose=0)

print("\n" + "="*50)
print("VALIDATION SET RESULTS")
print("="*50)
print(f"Loss: {val_loss:.4f}")
print(f"Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print("="*50)

## 16. Evaluate on Test Set

In [ ]:
# Evaluate on test set with detailed diagnostics
print("\nEvaluating on Test Set...")
test_loss, test_accuracy = model.evaluate(test_inputs, test_labels, batch_size=32, verbose=0)

print("\n" + "="*50)
print("TEST SET RESULTS")
print("="*50)
print(f"Loss: {test_loss:.4f}")
print(f"Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Additional diagnostics
print(f"\nTest Set Size: {len(test_labels)} samples")
print(f"  HUMAN samples: {np.sum(test_labels == 0)} ({np.sum(test_labels == 0)/len(test_labels)*100:.1f}%)")
print(f"  AI samples: {np.sum(test_labels == 1)} ({np.sum(test_labels == 1)/len(test_labels)*100:.1f}%)")

# Check for overfitting
if 'val_accuracy' in history.history:
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    acc_gap = final_train_acc - final_val_acc
    print(f"\nOverfitting Check:")
    print(f"  Final Train Accuracy: {final_train_acc:.4f}")
    print(f"  Final Val Accuracy: {final_val_acc:.4f}")
    print(f"  Accuracy Gap: {acc_gap:.4f}")
    if acc_gap > 0.05:
        print(f"  ⚠️ Warning: Possible overfitting detected (gap > 5%)")
    else:
        print(f"  ✓ No significant overfitting detected")

print("="*50)

## 17. Plot Training History

In [ ]:
# Create results directory
os.makedirs('results', exist_ok=True)

# Plot training history
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2.5, marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2.5, marker='s')
plt.title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2.5, marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2.5, marker='s')
plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/sinbert_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training history plot saved to results/sinbert_training_history.png")

## 18. Generate Predictions on Test Set

In [ ]:
# Generate predictions
print("Generating predictions on test set...")
predictions = model.predict(test_inputs, batch_size=32, verbose=0)

# Get predicted labels
predicted_labels = np.argmax(predictions.logits, axis=1)

# Get prediction probabilities
probabilities = tf.nn.softmax(predictions.logits).numpy()
positive_class_probs = probabilities[:, 1]  # Probability for AI class

print(f"✓ Predictions generated for {len(predicted_labels)} samples")
print(f"\nPrediction distribution:")
print(f"  HUMAN: {np.sum(predicted_labels == 0)}")
print(f"  AI: {np.sum(predicted_labels == 1)}")

## 19. Confusion Matrix

In [ ]:
# Generate confusion matrix with detailed analysis
cm = confusion_matrix(test_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.set(font_scale=1.2)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=['HUMAN', 'AI']
)
disp.plot(cmap='Blues', values_format='d', ax=plt.gca())

plt.title('Confusion Matrix - SinBERT Sinhala Classifier', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.savefig('results/sinbert_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Confusion matrix saved to results/sinbert_confusion_matrix.png")

# Print confusion matrix values with percentages
total = cm.sum()
tn, fp, fn, tp = cm.ravel()

print("\n" + "="*60)
print("Confusion Matrix Analysis")
print("="*60)
print(f"True Negatives (HUMAN→HUMAN): {tn:4d} ({tn/total*100:.1f}%)")
print(f"False Positives (HUMAN→AI):   {fp:4d} ({fp/total*100:.1f}%)")
print(f"False Negatives (AI→HUMAN):   {fn:4d} ({fn/total*100:.1f}%)")
print(f"True Positives (AI→AI):       {tp:4d} ({tp/total*100:.1f}%)")
print("="*60)

# Calculate error rates
print(f"\nError Analysis:")
print(f"  False Positive Rate: {fp/(fp+tn):.4f} ({fp/(fp+tn)*100:.2f}%)")
print(f"  False Negative Rate: {fn/(fn+tp):.4f} ({fn/(fn+tp)*100:.2f}%)")
print(f"  Misclassification Rate: {(fp+fn)/total:.4f} ({(fp+fn)/total*100:.2f}%)")

# Identify most common error
if fp > fn:
    print(f"  ⚠️ Model tends to over-predict AI (more false positives)")
elif fn > fp:
    print(f"  ⚠️ Model tends to under-predict AI (more false negatives)")
else:
    print(f"  ✓ Balanced error distribution")

## 20. ROC Curve and AUC Score

In [ ]:
# Calculate ROC curve and AUC with improved diagnostics
from sklearn.preprocessing import label_binarize

# Ensure probabilities are numpy arrays
if hasattr(positive_class_probs, 'numpy'):
    probs_np = positive_class_probs.numpy()
else:
    probs_np = np.array(positive_class_probs)

# Clip probabilities to valid range
probs_np = np.clip(probs_np, 0, 1)

# For binary classification, determine which class is AI
ai_class_idx = label_mapping['AI']  # Should be 1
human_class_idx = label_mapping['HUMAN']  # Should be 0

print(f"AI index: {ai_class_idx}, HUMAN index: {human_class_idx}")
print(f"Probability range: [{probs_np.min():.6f}, {probs_np.max():.6f}]")
print(f"Mean prob when true=AI:    {probs_np[test_labels == ai_class_idx].mean():.6f}")
print(f"Mean prob when true=HUMAN: {probs_np[test_labels == human_class_idx].mean():.6f}")
print(f"Unique probability values: {len(np.unique(probs_np))}")

# Calculate ROC curve using AI as positive class
fpr, tpr, thresholds = roc_curve(
    y_true=test_labels,
    y_score=probs_np,
    pos_label=ai_class_idx
)
auc_score = roc_auc_score(test_labels, probs_np)

# Plot ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, label=f'SinBERT (AUC = {auc_score:.4f})', linewidth=3, color='#2E86DE')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=2, label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - SinBERT Sinhala Classifier', fontsize=14, fontweight='bold', pad=20)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.tight_layout()
plt.savefig('results/sinbert_roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ ROC curve saved to results/sinbert_roc_curve.png")
print(f"\nAUC Score: {auc_score:.4f}")

# Additional diagnostics
print(f"\nROC Curve Diagnostics:")
print(f"  Number of thresholds: {len(thresholds)}")
print(f"  Threshold range: [{thresholds.min():.6f}, {thresholds.max():.6f}]")
print(f"  Best threshold (Youden's J): {thresholds[np.argmax(tpr - fpr)]:.6f}")

## 21. Precision-Recall Curve

In [ ]:
# Calculate precision-recall curve
from sklearn.metrics import average_precision_score

precision, recall, pr_thresholds = precision_recall_curve(test_labels, positive_class_probs)
avg_precision = average_precision_score(test_labels, positive_class_probs)

# Plot precision-recall curve
plt.figure(figsize=(10, 8))
plt.plot(recall, precision, linewidth=3, color='#10AC84', label=f'SinBERT (AP = {avg_precision:.4f})')
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve - SinBERT Sinhala Classifier', fontsize=14, fontweight='bold', pad=20)
plt.legend(loc='lower left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/sinbert_precision_recall_curve.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Precision-Recall curve saved to results/sinbert_precision_recall_curve.png")
print(f"\nAverage Precision Score: {avg_precision:.4f}")

## 22. Classification Report

In [ ]:
# Generate classification report
report = classification_report(
    test_labels,
    predicted_labels,
    target_names=['HUMAN', 'AI'],
    digits=4
)

print("\n" + "="*70)
print("CLASSIFICATION REPORT - TEST SET")
print("="*70)
print(report)
print("="*70)

# Save report to file
with open('results/sinbert_classification_report.txt', 'w') as f:
    f.write("="*70 + "\n")
    f.write("CLASSIFICATION REPORT - SinBERT Sinhala Classifier\n")
    f.write("="*70 + "\n\n")
    f.write(report)
    f.write("\n" + "="*70)

print("\n✓ Classification report saved to results/sinbert_classification_report.txt")

## 23. Detailed Performance Metrics

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Calculate metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels, average='weighted')
recall = recall_score(test_labels, predicted_labels, average='weighted')
f1 = f1_score(test_labels, predicted_labels, average='weighted')

# Class-specific metrics
precision_per_class = precision_score(test_labels, predicted_labels, average=None)
recall_per_class = recall_score(test_labels, predicted_labels, average=None)
f1_per_class = f1_score(test_labels, predicted_labels, average=None)

# Create summary DataFrame
metrics_summary = pd.DataFrame({
    'Metric': [
        'Test Accuracy',
        'Test Loss',
        'AUC Score',
        'Average Precision',
        'Weighted Precision',
        'Weighted Recall',
        'Weighted F1-Score',
        'HUMAN - Precision',
        'HUMAN - Recall',
        'HUMAN - F1-Score',
        'AI - Precision',
        'AI - Recall',
        'AI - F1-Score'
    ],
    'Value': [
        f'{accuracy:.4f}',
        f'{test_loss:.4f}',
        f'{auc_score:.4f}',
        f'{avg_precision:.4f}',
        f'{precision:.4f}',
        f'{recall:.4f}',
        f'{f1:.4f}',
        f'{precision_per_class[0]:.4f}',
        f'{recall_per_class[0]:.4f}',
        f'{f1_per_class[0]:.4f}',
        f'{precision_per_class[1]:.4f}',
        f'{recall_per_class[1]:.4f}',
        f'{f1_per_class[1]:.4f}'
    ]
})

print("\n" + "="*60)
print("DETAILED PERFORMANCE METRICS")
print("="*60)
print(metrics_summary.to_string(index=False))
print("="*60)

# Save metrics
metrics_summary.to_csv('results/sinbert_performance_metrics.csv', index=False)
print("\n✓ Metrics saved to results/sinbert_performance_metrics.csv")

## 24. Test Model on Sample Texts

In [ ]:
def predict_text(text, model, tokenizer, max_length=256):
    """
    Predict if a given text is HUMAN or AI-generated.
    Returns prediction and confidence score.
    """
    # Preprocess text
    processed_text = preprocess_sinhala_text(text)

    # Tokenize
    encoding = tokenizer(
        [processed_text],
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='tf'
    )

    # Predict
    outputs = model(encoding)
    probs = tf.nn.softmax(outputs.logits, axis=1).numpy()[0]

    predicted_label = np.argmax(probs)
    confidence = probs[predicted_label]

    return {
        'label': reverse_mapping[predicted_label],
        'confidence': float(confidence),
        'human_prob': float(probs[0]),
        'ai_prob': float(probs[1])
    }

# Test on sample texts from test set
print("\n" + "="*70)
print("SAMPLE PREDICTIONS")
print("="*70)

sample_indices = np.random.choice(len(test_df), 5, replace=False)

correct_predictions = 0
for i, idx in enumerate(sample_indices, 1):
    text = test_df.iloc[idx]['text']
    true_label = test_df.iloc[idx]['label']

    result = predict_text(text, model, tokenizer, MAX_LENGTH)
    is_correct = result['label'] == true_label
    correct_predictions += is_correct

    status = "✓" if is_correct else "✗"
    print(f"\n{status} Sample {i}:")
    print(f"Text: {text[:100]}...")
    print(f"True Label: {true_label}")
    print(f"Predicted: {result['label']} (Confidence: {result['confidence']:.2%})")
    print(f"  HUMAN: {result['human_prob']:.2%} | AI: {result['ai_prob']:.2%}")
    print("-" * 70)

print(f"\nSample Accuracy: {correct_predictions}/{len(sample_indices)} ({correct_predictions/len(sample_indices)*100:.1f}%)")

## 25. Model Summary and Comparison

In [ ]:
# Final summary
print("\n" + "="*70)
print("SinBERT SINHALA CLASSIFIER - FINAL SUMMARY")
print("="*70)
print(f"\nModel: {MODEL_NAME}")
print(f"Parameters: {model.num_parameters():,}")
print(f"Max Sequence Length: {MAX_LENGTH}")
print(f"\nDataset Sizes:")
print(f"  Training: {len(train_df):,} samples")
print(f"  Validation: {len(val_df):,} samples")
print(f"  Test: {len(test_df):,} samples")
print(f"\nTraining Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {TRAIN_BATCH_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"\nPerformance:")
print(f"  Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  AUC Score: {auc_score:.4f}")
print(f"  Average Precision: {avg_precision:.4f}")
print(f"  F1-Score: {f1:.4f}")
print(f"\nModel saved to: {MODEL_SAVE_PATH}")
print("="*70)

print("\n✓ All tasks completed successfully!")
print("\nNote: SinBERT-large is specifically trained on Sinhala data, which may provide")
print("better performance compared to multilingual models like mBERT for Sinhala text.")

## 26. Download Trained Model (Optional)

In [ ]:
import shutil
from google.colab import files

# Define paths
folder_path = 'models/sinbert_sinhala_classifier'
output_filename = 'sinbert_sinhala_classifier'

# Zip the folder
shutil.make_archive(output_filename, 'zip', folder_path)
print(f"Zipped {folder_path} to {output_filename}.zip")

# Download the zip file
files.download(f"{output_filename}.zip")